# Auto Labeller - A step by step guide
Here we provide a step by step guide for you to experiment with the semi auto-labelling tool.

## Step 1 - Standard Library Imports
First lets perform the standard library imports.

In [1]:
# Standard Libary Imports
import pandas as pd
import numpy as np

from src.toolkit.plot import *
from src.toolkit.autolabel import *

from sklearn.naive_bayes import MultinomialNB

## Step 2 - Adjust the Input
Input the file path to your text data.

The tabular data should have the following format

<img style="float: left;" src="data/images/input_sample.png">

In [2]:
# file path to text data
text_path = "data/movies500.csv"  ##  PUT YOUR FILEPATH

# name of column that contains text data
text_column_name = "overview"  ##  PUT YOUR TEXT COLUMN

In [3]:
data = pd.read_csv(text_path)[[text_column_name]]

In [4]:
data.head(5)  ## CHECK THE FORMAT OF YOUR DATA

overview
0  A family wedding reignites the ancient feud be...
1  Cheated on, mistreated and stepped on, the wom...
2  Obsessive master thief, Neil McCauley leads a ...
3  An ugly duckling having undergone a remarkable...
4  A mischievous young boy, Tom Sawyer, witnesses...

## Step 3 - Data Preprocessing
Run the following code to prepare your text data for model input.

In [6]:
corpus = data[text_column_name]

preprocessor = Preprocessor()

# Text Preprocessing
stopwords_path = "data/stopwords.csv"  ## STOPWORDS PATH
preprocessed_corpus = preprocessor.corpus_preprocess(corpus=corpus, stopwords_path=stopwords_path)

# Replace bigrams
data[text_column_name] = preprocessor.corpus_replace_bigrams(corpus=preprocessed_corpus, min_df=50, max_df=500)

## Step 4 - Create a labels Dictionary
Open your favourite tabular editor (excel/Numbers/libre) to create a labels csv file based on the suggested vocabulary shown below.

The label dictionary should be in a tabular format with keys that relates to each label

**Note: these labels needs to exist within the dataset**

<img style="float: left;" src="data/images/labels_sample.png"> </img>

In [7]:
n_words = 20  # CHANGE THE NUMBER OF RECOMMENDED WORDS (IF YOU WANT TO)

# Returns a matrix of recommended words
topic_model, dtm, best_n = recommend_words(corpus) 
topic_model.show_topics(dtm=dtm, best_n=best_n, n_words=n_words)

0        1           2          3          4          5   \
0        man    demon    convince      story  discovers  political   
1       love     fall      french    student       king        two   
2      young  husband         try    officer  beautiful      story   
3     killer   serial   detective        law     female     victim   
4   daughter     wife        face      leave     suffer        die   
5       team     stop        evil        try     former        cop   
6      woman     town        come     pursue      small  notorious   
7       film    story        work       tell       base        red   
8        get    thing      friend         go       best      marry   
9       find      new         way       time        hit        job   
10      take      one        meet     become       plan        job   
11  american      war  expatriate      paris     affair      world   
12    family   father      search       year      irish        son   
13      drug     want        name  operation       lord      local   
14      life     move       learn  housewife     friend       four   
15    school     girl        high     parent    teacher         go   
16  criminal   police      leader       kill        run       know   
17    murder      two         run     accuse    project        put   

            6           7           8            9             10  \
0        guest      friend        true         like          look   
1         play    romantic       story           dr   counterpart   
2    dangerous        help         boy         home        escape   
3     violence      person         gay          one          lead   
4          car    accident      second         poor        castle   
5     computer       agent         kgb      protect         steal   
6    beautiful  mysterious        work       comedy         along   
7       comedy         set         boy      strange         world   
8         tell         old        help     marriage     boyfriend   
9       secret         die      friend         make           mob   
10      decide     involve       night          son         large   
11          ii      arrive      france         meet       control   
12    neighbor       widow    marriage         farm  relationship   
13        sell         cia  undercover        agent        prison   
14         day         end     lesbian         year          best   
15        trip     society        come         city       teenage   
16       group         old       money       kidnap          gang   
17  girlfriend       crime         boy  investigate          deal   

              11        12       13           14          15          16  \
0           know      fear  pretend       nature    confront     descend   
1       exchange     marry  brother         tale      famous  mysterious   
2   relationship       war     girl          men        name      doctor   
3            two       duo     clue  personality  journalist        past   
4           rule      need   become        begin         two        look   
5          world    rescue  russian         plan         use        chip   
6     successful      show     move         many     compete       dream   
7          bring     novel    movie       studio     village   adventure   
8           hire       cop   mother     business         try     brother   
9          bring       bos     work  advertising     ancient       human   
10          case       new  someone    blackmail       power        kill   
11        fellow     fight    civil     struggle      writer       south   
12         wound      face    local   adaptation        fear     captain   
13        dealer      work     help       police         eye        lead   
14     apartment      make   invite     threaten        good     student   
15           age      gang      new          mrs        look       child   
16          pose  mountain     rich         cult     revenge        lead   

## Step 5 - Feed your label dictionary
Input the filepath to your label dictionary here

In [14]:
labels_path = "data/labels.csv"  # INPUT PATH TO LABELS DICTIONARY
labels = pd.read_csv(labels_path)

In [9]:
# show label data
labels.head(5)  

Action   Romance Science Fiction   Thriller       War   Western
0  criminal      love         science     killer       war  criminal
1      kill    french             NaN  detective     world       NaN
2    kidnap  romantic             NaN        law        ii       NaN
3      gang     marry             NaN     victim  struggle       NaN
4   revenge  marriage             NaN        cop       NaN       NaN

## Step 6 - Enriching Labels
In this step, the model enriches your current labels with other words close to the current topic.

**NOTE: The first table shows your current label dictionary, the second shows the enriched dictionary. This enriched dictionary may be used for the topic labelling**

In [10]:
# current labels
labels.head(10)  

Action    Romance Science Fiction   Thriller       War   Western
0  criminal       love         science     killer       war  criminal
1      kill     french             NaN  detective     world       NaN
2    kidnap   romantic             NaN        law        ii       NaN
3      gang      marry             NaN     victim  struggle       NaN
4   revenge   marriage             NaN        cop       NaN       NaN
5   violent  boyfriend             NaN      agent       NaN       NaN
6       NaN       wife             NaN        kgb       NaN       NaN

In [11]:
autoLabeller = AutoLabeller(labels, corpus, data)
enriched_labels = autoLabeller.train()

enriched_labels  ## Enriched labels

Action    Romance Science Fiction   Thriller            War    Western
0    romantic   romantic           marry        law             ii        law
1       marry      marry         science       gold  disappearance       wife
2     married     modern            wife   renegade          chase       case
3        fall    science       terrorist    hostage           fall   attorney
4        wife       wife            case  terrorist        holiday       late
5     violent       fall           havoc   attorney           late     killer
6      french     french         nuclear    missile         french    revenge
7   operation    compete         america     killer       struggle       clue
8        love       idea         resolve       chip         refuse    retired
9      kidnap  notorious            army  operation         singer       next
10    revenge       love        intrigue    nuclear           jazz     serial
11      grows      bunch            side      civil       persuade   murderer
12    resolve    america       different    russian           nazi  detective
13   marriage     center        personal  detective          paris        cop
14    intense  introduce            view        cop     expatriate     assign
15      clean       nazi          assign      clean      bodyguard   criminal
16   criminal     serial       challenge       team            war  innocence
17    yearold   marriage          reason     design            bar       team
18      throw   intrigue          comedy      agent           club     danger
19     street       view          danger      sight          world     victim
20  boyfriend        war          attend     victim          slave       kill
21       gang  challenge            None        kgb           None       None
22       kill     comedy            None      world           None       None
23       None     attend            None       kill           None       None
24       None  boyfriend            None       None           None       None
25       None      world            None       None           None       None

## Step 7 - Apply the auto labeller
Input file path to output your labelled dataset

In [12]:
labelled_path = "data/movies500_labelled.csv"  # INPUT YOUR PREFERED OUTPUT PATH

In [13]:
mnb = MultinomialNB()
ypred = autoLabeller.apply(mnb)
ypred.to_csv(labelled_path)